<a href="https://colab.research.google.com/github/jdilger/useful_ee_scripts/blob/main/PlanetDownloaderForMosaics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
from pprint import pprint

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
import ee

ee.Authenticate()
ee.Initialize()

In [ ]:
API = 'key here'
mosaics = !curl https://api.planet.com/basemaps/v1/mosaics?api_key=$API
mosaic_jsons = json.loads(mosaics[0])['mosaics']
print(mosaic_jsons[0])

In [ ]:
# check date to see which mosaic you want
list_of_dates = list(map(lambda m: m['first_acquired'],mosaic_jsons))
print(list(map(lambda m: m['first_acquired'],mosaic_jsons)))
print(list_of_dates.index("2020-09-01T00:00:00.000Z"))


In [ ]:
# selecting 2019-12-01T00:00:00.000Z @ position 8 
index_select = "2020-09-01T00:00:00.000Z"
moi = mosaic_jsons[list_of_dates.index(index_select)]
pprint(moi)
start_date = moi['first_acquired'][0:10]
end_date = moi['last_acquired'][0:10]
print(start_date, end_date)

In [ ]:
# bbox TerraBio pilot site
lx = -53.461751914154526
ux = -50.961740478449897
ly = -6.942135599911456
uy = -5.442128738488679

quads = moi['_links']['quads']
iname = moi['name']
print(quads,iname)

quad_formatted = quads.replace('{lx}',str(lx))\
                      .replace('{ly}',str(ly))\
                      .replace('{ux}',str(ux))\
                      .replace('{uy}',str(uy))
print(quad_formatted)

In [ ]:
quad_fetch = requests.get(quad_formatted)
quad_fetch = quad_fetch.json()
pprint(quad_fetch['items'][0])

images = []
next = quad_formatted
while next:
  next_fetch = requests.get(next).json()
  images.extend(next_fetch['items'])
  # pprint(next_fetch['_links'])
  try:
    next = next_fetch['_links']['_next']
  except Exception as e:
    next = False

pprint('firest image :',images[0])
print('number of images :',len(images))

In [ ]:
downloadurls = list(map(lambda quad: [quad['id'],quad['_links']['download']], images))
downloadurls.sort()
# downloadurls[0]
# # check the number of images to download
# print(len(downloadurls))
# print(downloadurls[143])

In [ ]:
# manually created metadata for selected date
coldict = {

    'default' : {
        'date_start' : str(start_date),
        'date_end': str(end_date),
        'cloud' : 'gs://gee-upload/',
        'ee_dst':'projects/servir-amazonia/TerraBio/ImageCollections/Planet/'

    }
}
choice = 'default'
datestr = coldict[choice]['date_start']
datestrend = coldict[choice]['date_end']
# bandnames = 'blue,green,red,nir,alpha'

In [ ]:
# range 0 to length of iamges to download
for i in range(0,len(downloadurls)):
  url = downloadurls[i][1]
  name = f"{downloadurls[i][0]}-{datestr}"

  # download the image and pipe it to a google coud bcket
  !wget {url} -O - | gsutil cp - {coldict['default']['cloud']}{datestr}-{datestrend}/{name}.tif
  # print(f"{coldict['default']['cloud']}{name}.tif",datestr, datestrend)